In [45]:
 import os
 arr = os.listdir("notes")
import glob

txtfiles = []
for file in glob.glob("./notes/*.md"):
    txtfiles.append(file)
    
arr = [md.replace(".md", "")for md in txtfiles]

In [46]:

arr_qid = []
for item in arr:
    if "Q" in item:
        arr_qid.append(item)
arr_qid = [md.replace("./notes/Q", "wd:Q")for md in arr_qid]

In [47]:
readings = "{"
for i in arr_qid:
    readings = readings + " " + i
readings = readings + " }"

print(readings)

{ wd:Q47579749 wd:Q24678240 wd:Q31194033 wd:Q29617753 wd:Q101226729 wd:Q90591926 wd:Q27021885 wd:Q35830509 wd:Q42778383 wd:Q37301036 wd:Q62924270 wd:Q28254676 wd:Q74657187 wd:Q37629171 wd:Q42585641 wd:Q27010194 wd:Q28709067 wd:Q102232524 wd:Q26707522 wd:Q56880438 wd:Q96817154 wd:Q42776227 wd:Q52720499 wd:Q28595082 wd:Q100525924 wd:Q34169535 wd:Q2703462 wd:Q38798014 wd:Q37490532 wd:Q60017992 wd:Q98568691 wd:Q28974693 wd:Q100395427 wd:Q38622289 wd:Q57011346 wd:Q28181838 wd:Q30570244 wd:Q28842753 wd:Q58084012 wd:Q28292614 wd:Q28596617 wd:Q33037209 wd:Q57462271 wd:Q88605805 wd:Q90109096 wd:Q28200510 wd:Q29615252 wd:Q98665291 wd:Q98622035 wd:Q91997728 wd:Q104092178 wd:Q98568692 wd:Q34065481 wd:Q102926772 wd:Q33926910 wd:Q79763269 wd:Q57774232 wd:Q38432885 wd:Q96032351 wd:Q101473819 wd:Q103964322 wd:Q55483834 wd:Q21092566 wd:Q49483648 wd:Q78658697 wd:Q46221687 wd:Q102542608 wd:Q99564826 wd:Q64097544 wd:Q27311570 wd:Q34878068 wd:Q86694255 wd:Q34094294 wd:Q52374237 wd:Q42255083 wd:Q51427139 wd

In [48]:
query_1 = """

#defaultView:Table
SELECT
  (MIN(?dates) AS ?date)
  ?work ?workLabel
  (GROUP_CONCAT(DISTINCT ?type_label; separator=", ") AS ?type)
  (SAMPLE(?pages_) AS ?pages)
  ?venue ?venueLabel
  (GROUP_CONCAT(DISTINCT ?author_label; separator=", ") AS ?authors)
WHERE {
  VALUES ?work """ +  readings + """.
  ?work wdt:P50 ?author .
  OPTIONAL {
    ?author rdfs:label ?author_label_ . FILTER (LANG(?author_label_) = 'en')
  }
  BIND(COALESCE(?author_label_, SUBSTR(STR(?author), 32)) AS ?author_label)
  OPTIONAL { ?work wdt:P31 ?type_ . ?type_ rdfs:label ?type_label . FILTER (LANG(?type_label) = 'en') }
  ?work wdt:P577 ?datetimes .
  BIND(xsd:date(?datetimes) AS ?dates)
  OPTIONAL { ?work wdt:P1104 ?pages_ }
  OPTIONAL { ?work wdt:P1433 ?venue }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,da,de,es,fr,jp,no,ru,sv,zh". }  
}
GROUP BY ?work ?workLabel ?venue ?venueLabel
ORDER BY DESC(?date)  

"""

with open("sparql/articles.rq", "w+") as f:
  f.write(query_1)

In [49]:
query_2 = """

#defaultView:BubbleChart
SELECT ?score ?topic ?topicLabel
WITH {
  SELECT
    (SUM(?score_) AS ?score)
    ?topic
  WHERE {
        {
      SELECT (100 AS ?score_) ?topic WHERE {
        VALUES ?work """ +  readings + """.
         ?work  wdt:P921 ?topic . 
      }
    }
    UNION
    {
      SELECT (1 AS ?score_) ?topic WHERE {
        VALUES ?work """ +  readings + """.
        ?citing_work wdt:P2860 ?work .
        ?citing_work wdt:P921 ?topic . 
      }
    }
  }
  GROUP BY ?topic
} AS %results 
WHERE {
  INCLUDE %results
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en,da,de,es,jp,no,ru,sv,zh". }
}
ORDER BY DESC(?score)
LIMIT 50

"""


with open("sparql/topic_bubble.rq", "w+") as f:
  f.write(query_2)

In [50]:
query_3 = """


#defaultView:Table
SELECT ?count ?theme ?themeLabel ?example_work ?example_workLabel
WITH {
  SELECT (COUNT(?work) AS ?count) ?theme (SAMPLE(?work) AS ?example_work)
  WHERE {
    VALUES ?work """ +  readings + """.
    ?work wdt:P921 ?theme .
  }
  GROUP BY ?theme
} AS %result
WHERE {
  INCLUDE %result
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,da,de,es,fr,jp,nl,no,ru,sv,zh" . } 
}
ORDER BY DESC(?count) 

"""

with open("sparql/topic_table.rq", "w+") as f:
  f.write(query_3)

In [51]:
query_4 = """


# Venue statistics for a collection
SELECT
  ?count (SAMPLE(?short_name_) AS ?short_name)
  ?venue ?venueLabel
  ?topics ?topicsUrl
WITH {
  SELECT
    (COUNT(DISTINCT ?work) as ?count)
    ?venue
    (GROUP_CONCAT(DISTINCT ?topic_label; separator=", ") AS ?topics)
  WHERE {
    VALUES ?work """ +  readings + """.
    ?work wdt:P1433 ?venue .
    OPTIONAL {
      ?venue wdt:P921 ?topic .
      ?topic rdfs:label ?topic_label . FILTER(LANG(?topic_label) = 'en') }
  }
  GROUP BY ?venue
} AS %result
WHERE {
  INCLUDE %result
  OPTIONAL { ?venue wdt:P1813 ?short_name_ . }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,da,de,es,fr,jp,nl,no,ru,sv,zh". }  
} 
GROUP BY ?count ?venue ?venueLabel ?topics ?topicsUrl
ORDER BY DESC(?count)

"""

with open("sparql/venues.rq", "w+") as f:
  f.write(query_4)

In [52]:
query_5 = """


#defaultView:Map
SELECT ?organization ?organizationLabel ?geo ?count ?layer
WITH {
  SELECT DISTINCT ?organization ?geo (COUNT(DISTINCT ?work) AS ?count) WHERE {
    VALUES ?work """ +  readings + """.
        ?work wdt:P50 ?author .
    ?author ( wdt:P108 | wdt:P463 | wdt:P1416 ) / wdt:P361* ?organization . 
    ?organization wdt:P625 ?geo .
  }
  GROUP BY ?organization ?geo ?count
  ORDER BY DESC (?count)
  LIMIT 2000
} AS %organizations
WHERE {
  INCLUDE %organizations
  BIND(IF( (?count < 1), "No results", IF((?count < 2), "1 result", IF((?count < 5), "1 < results ≤ 10", IF((?count < 101), "10 < results ≤ 100", IF((?count < 1001), "100 < results ≤ 1000", IF((?count < 10001), "1000 < results ≤ 10000", "10000 or more results") ) ) ) )) AS ?layer )
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }        
 }
ORDER BY DESC (?count)


"""


with open("sparql/author_map.rq", "w+") as f:
  f.write(query_5)

In [53]:
query_6 = """


#defaultView:Table
SELECT
  ?count
  ?citing_author ?citing_authorLabel

  # Either show the ORCID iD or construct part of a URL to search on the ORCID homepage
  (COALESCE(?orcid_, CONCAT("orcid-search/quick-search/?searchQuery=", ENCODE_FOR_URI(?citing_authorLabel))) AS ?orcid)
WITH {
  SELECT (COUNT(?citing_work) AS ?count) ?citing_author WHERE {
    VALUES ?work """ +  readings + """.
    ?citing_work wdt:P2860 ?work . 
    ?citing_work wdt:P50 ?citing_author .
  }
  GROUP BY ?citing_author 
  ORDER BY DESC(?count)
  LIMIT 500
} AS %counts
WITH {
  # An author might have multiple ORCID iDs
  SELECT
    ?count
    ?citing_author
    (SAMPLE(?orcids) AS ?orcid_)
  WHERE {
    INCLUDE %counts
    OPTIONAL { ?citing_author wdt:P496 ?orcids }
  }
  GROUP BY ?count ?citing_author
} AS %result
WHERE {
  INCLUDE %result
  
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,da,de,es,fr,jp,nl,no,ru,sv,zh". }
} 
ORDER BY DESC(?count)


"""

with open("sparql/citing_authors.rq", "w+") as f:
  f.write(query_6)

In [54]:
query_1 = """#defaultView:Table
SELECT
  (MIN(?dates) AS ?date)
  ?work ?workLabel
  (GROUP_CONCAT(DISTINCT ?type_label; separator=", ") AS ?type)
  (SAMPLE(?pages_) AS ?pages)
  ?venue ?venueLabel
  (GROUP_CONCAT(DISTINCT ?author_label; separator=", ") AS ?authors)
WHERE {
  VALUES ?work """ +  readings + """.
  ?work wdt:P50 ?author .
  OPTIONAL {
    ?author rdfs:label ?author_label_ . FILTER (LANG(?author_label_) = 'en')
  }
  BIND(COALESCE(?author_label_, SUBSTR(STR(?author), 32)) AS ?author_label)
  OPTIONAL { ?work wdt:P31 ?type_ . ?type_ rdfs:label ?type_label . FILTER (LANG(?type_label) = 'en') }
  ?work wdt:P577 ?datetimes .
  BIND(xsd:date(?datetimes) AS ?dates)
  OPTIONAL { ?work wdt:P1104 ?pages_ }
  OPTIONAL { ?work wdt:P1433 ?venue }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en,da,de,es,fr,jp,no,ru,sv,zh". }  
}
GROUP BY ?work ?workLabel ?venue ?venueLabel
ORDER BY DESC(?date)  

"""

In [56]:
import urllib 
test = "https://query.wikidata.org/embed.html#" + urllib.parse.quote(query_1, safe='')
print(test)

https://query.wikidata.org/embed.html#%23defaultView%3ATable%0ASELECT%0A%20%20%28MIN%28%3Fdates%29%20AS%20%3Fdate%29%0A%20%20%3Fwork%20%3FworkLabel%0A%20%20%28GROUP_CONCAT%28DISTINCT%20%3Ftype_label%3B%20separator%3D%22%2C%20%22%29%20AS%20%3Ftype%29%0A%20%20%28SAMPLE%28%3Fpages_%29%20AS%20%3Fpages%29%0A%20%20%3Fvenue%20%3FvenueLabel%0A%20%20%28GROUP_CONCAT%28DISTINCT%20%3Fauthor_label%3B%20separator%3D%22%2C%20%22%29%20AS%20%3Fauthors%29%0AWHERE%20%7B%0A%20%20VALUES%20%3Fwork%20%7B%20wd%3AQ47579749%20wd%3AQ24678240%20wd%3AQ31194033%20wd%3AQ29617753%20wd%3AQ101226729%20wd%3AQ90591926%20wd%3AQ27021885%20wd%3AQ35830509%20wd%3AQ42778383%20wd%3AQ37301036%20wd%3AQ62924270%20wd%3AQ28254676%20wd%3AQ74657187%20wd%3AQ37629171%20wd%3AQ42585641%20wd%3AQ27010194%20wd%3AQ28709067%20wd%3AQ102232524%20wd%3AQ26707522%20wd%3AQ56880438%20wd%3AQ96817154%20wd%3AQ42776227%20wd%3AQ52720499%20wd%3AQ28595082%20wd%3AQ100525924%20wd%3AQ34169535%20wd%3AQ2703462%20wd%3AQ38798014%20wd%3AQ37490532%20wd%3AQ60017992%2

In [62]:

url1 = "https://query.wikidata.org/embed.html#" + urllib.parse.quote(query_1, safe='')
url2 = "https://query.wikidata.org/embed.html#" + urllib.parse.quote(query_2, safe='')
url3 = "https://query.wikidata.org/embed.html#" + urllib.parse.quote(query_3, safe='')
url4 = "https://query.wikidata.org/embed.html#" + urllib.parse.quote(query_4, safe='')
url5 = "https://query.wikidata.org/embed.html#" + urllib.parse.quote(query_5, safe='')
url6 = "https://query.wikidata.org/embed.html#" + urllib.parse.quote(query_6, safe='')

html = """


<!DOCTYPE html>
<html>
<head>
	<meta charset="utf-8">
	<meta name="viewport" content="width=device-width, initial-scale=1">
	<title>Wikidata Bib - Tiago Lubiana</title>
	<meta property="og:description" content="powered by Wikidata">
	<link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bulma@0.8.2/css/bulma.min.css">
	<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css">
</head>
	<body>
	<section class="section">
    		<div class="container">
			<div class="columns is-centered">
				<div class="column is-half has-text-centered">
      					<h1 class="title is-1"> My Readings</h1>
    				</div>
			</div>
		</div>
		<div class="column is-half has-text-centered">
  	</section>
	<div class="container">
			<div class="columns is-centered">
				<div class="column is-half has-text-centered">
					<h4 class="subtitle is-4">
       					Dashboard of Tiago Lubiana's Readings
      					</p>
				</div>
			</div>
	</div>
	</br>
	</br>
	<section class="section extra">
		<div class="container" id="query">
			<div class="columns is-centered">
   				 <div class="tabs is-centered">
    					<ul>
      						<li id="one-tab" class="">
        							<a onclick="switchToOne()">
      	 							<span>Articles</span> 
      							</a>
      						</li>
      						<li id="two-tab">
        							<a onclick="switchToTwo()">
										<span>Topic Bubble</span>
      							</a>
                            </li>
                            <li id="three-tab">
                                <a onclick="switchToThree()">
                                    <span>Topic Count</span>
                              </a>
                          </li>
                          <li id="four-tab">
                            <a onclick="switchToFour()">
                                <span>Venues</span>
                          </a>
                      </li>
                      <li id="five-tab">
                        <a onclick="switchToFive()">
                            <span>Author Map</span>
                      </a>
                      <li id="six-tab">
                        <a onclick="switchToSix()">
                            <span>Citing Authors</span>
                      </a>
                  </li>  
    					</ul>
    					<!--/tabs is-centered-->
  				</div>
 			</div>
		</div>
    </section>
	<div class="container">
    		<div id="one-tab-content">
      			<h5 class="title is-5" style="text-align:center;"> What I've been reading </h5>
				<div class="columns is-centered"
					<p style="text-align: center">
						<iframe width=92% height="500" src=""" + '"'+ url1 +'"' + """></iframe>
					</p>
				</div>
    			</div>
    		<div class="is-hidden" id="two-tab-content">
      			<<h5 class="title is-5" style="text-align:center;"> Topics I've been reading in bubbles </h5>
				<div class="columns is-centered"
					<p style="text-align: center">
						<iframe width=92% height="500" src=""" + '"'+ url2 +'"' + """></iframe>
					</p>
				</div>
                </div>
            <div class="is-hidden" id="three-tab-content">
                    <<h5 class="title is-5" style="text-align:center;"> Topics I've been reading in a table </h5>
                  <div class="columns is-centered"
                      <p style="text-align: center">
                        <iframe width=92% height="500" src=""" + '"'+ url3+'"' + """></iframe>
                      </p>
                  </div>
                  </div>
            <div class="is-hidden" id="four-tab-content">
                    <<h5 class="title is-5" style="text-align:center;">  Which journals I've been reading</h5>
                  <div class="columns is-centered"
                      <p style="text-align: center">
                        <iframe width=92% height="500" src=""" + '"'+ url4 +'"' + """></iframe>
                      </p>
                  </div>
                  </div>
            <div class="is-hidden" id="five-tab-content">
                    <<h5 class="title is-5" style="text-align:center;"> Where are the authors I've been reading </h5>
                  <div class="columns is-centered"
                      <p style="text-align: center">
                        <iframe width=92% height="500" src=""" + '"'+ url5 +'"' + """></iframe>
                      </p>
                  </div>
                  </div>
            <div class="is-hidden" id="six-tab-content">
                    <<h5 class="title is-5" style="text-align:center;"> Who cites what I've been reading </h5>
                  <div class="columns is-centered"
                      <p style="text-align: center">
                        <iframe width=92% height="500" src=""" + '"'+ url6 +'"' + """></iframe>
                      </p>
                  </div>
                  </div>
  	</div>
  <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.2.1/jquery.min.js"></script>
  <script>

    function switchToOne() {
      removeActive();
      hideAll();
      $("#one-tab").addClass("is-active");
      $("#one-tab-content").removeClass("is-hidden");
    }

    function switchToTwo() {
      removeActive();
      hideAll();
      $("#two-tab").addClass("is-active");
      $("#two-tab-content").removeClass("is-hidden");
    }

    function switchToThree() {
      removeActive();
      hideAll();
      $("#three-tab").addClass("is-active");
      $("#three-tab-content").removeClass("is-hidden");
    }

    function switchToFour() {
      removeActive();
      hideAll();
      $("#four-tab").addClass("is-active");
      $("#four-tab-content").removeClass("is-hidden");
    }

    function switchToFive() {
      removeActive();
      hideAll();
      $("#five-tab").addClass("is-active");
      $("#five-tab-content").removeClass("is-hidden");
    }

    function switchToSix() {
      removeActive();
      hideAll();
      $("#six-tab").addClass("is-active");
      $("#six-tab-content").removeClass("is-hidden");
    }

    function removeActive() {
      $("li").each(function() {
        $(this).removeClass("is-active");
      });
    }

    function hideAll(){
      $("#one-tab-content").addClass("is-hidden");
      $("#two-tab-content").addClass("is-hidden");
      $("#three-tab-content").addClass("is-hidden");
      $("#four-tab-content").addClass("is-hidden");
      $("#five-tab-content").addClass("is-hidden");
      $("#six-tab-content").addClass("is-hidden");
    }
	
  </script>
  </br>

<footer class="footer">
  <div class="container">
    <div class="content has-text-centered">
      <p>
        The content is licensed <a href="https://creativecommons.org/publicdomain/zero/1.0/">under Creative Commons CC0</a>
     
        <p> Wikidata SPARQL Queries adapted from <a href="https://scholia.toolforge.org/">Scholia</a>  </p>
        <p> <strong>Dashboard</strong> adapted by <a href="https://www.wikidata.org/wiki/User:TiagoLubiana">TiagoLubiana</a>
	  from the <a href="https://wikiproject-india.github.io/covid19dashboard/">COVID-19 in India dashboard </a>
      </p>
    </div>
  </div>
</footer>
</body>
</html>
"""

In [63]:
with open("docs/test.html", "w") as f:
    f.write(html)